# Assignment 11

**Recommendation System**


**1. Data Preprocessing:**

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

In [5]:
df=pd.read_csv("anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [6]:
df.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [8]:
df = df.dropna()

In [9]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [10]:
df.describe()

,anime_id,rating,members
count,12017.000000,12017.000000,1.201700e+04
mean,13638.001165,6.478264,1.834888e+04
std,11231.076675,1.023857,5.537250e+04
min,1.000000,1.670000,1.200000e+01
25%,3391.000000,5.890000,2.250000e+02
50%,9959.000000,6.570000,1.552000e+03
75%,23729.000000,7.180000,9.588000e+03
max,34519.000000,10.000000,1.013917e+06


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [27]:
df['type'] = df['type'].fillna('None')
df['genre'] = df['genre'].fillna('None')
df['rating'] = df['rating'].fillna(df['rating'].median())
episode = df['episodes'].replace('Unknown', np.nan)

In [12]:
df.shape

(12017, 7)

**2. Feature Extraction:**

Convert Categorical Features

In [24]:
type_ = pd.get_dummies(df['type'])
genre_ = df['genre'].str.get_dummies(sep=',')
genre_.head()

,' Adventure',' Adventure'],' Cars',' Cars'],' Comedy',' Comedy'],' Dementia',' Dementia'],' Demons',' Demons'],...,['Shounen'],['Slice of Life',['Slice of Life'],['Space'],['Sports'],['Super Power',['Supernatural'],['Thriller'],['Vampire'],['Yaoi']
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
ccf = pd.concat([genre_, type_, episode, df['rating'], df['members']],axis=1)
ccf.head()

,' Adventure',' Adventure'],' Cars',' Cars'],' Comedy',' Comedy'],' Dementia',' Dementia'],' Demons',' Demons'],...,['Yaoi'],Movie,Music,ONA,OVA,Special,TV,episodes,rating,members
0,0,0,0,0,0,0,0,0,0,0,...,0,True,False,False,False,False,False,1,0.924370,0.197867
1,1,0,0,0,0,0,0,0,0,0,...,0,False,False,False,False,False,True,64,0.911164,0.782769
2,0,0,0,0,1,0,0,0,0,0,...,0,False,False,False,False,False,True,51,0.909964,0.112683
3,0,0,0,0,0,0,0,0,0,0,...,0,False,False,False,False,False,True,24,0.900360,0.664323
4,0,0,0,0,1,0,0,0,0,0,...,0,False,False,False,False,False,True,51,0.899160,0.149180


Normalization

In [36]:
from sklearn.preprocessing import MinMaxScaler
scaled = MinMaxScaler()
ccf= scaled.fit_transform(ccf)
ccf

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.24369748e-01, 1.97866664e-01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.46725371e-02, 9.11164466e-01, 7.82768603e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.75178866e-02, 9.09963986e-01, 1.12683141e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.65107320e-03, 3.85354142e-01, 2.04161139e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.97358944e-01, 1.60764569e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.54981993e-01, 1.28217141e-04]])

In [37]:
df[['rating', 'members']] = scaler.fit_transform(df[['rating', 'members']])

**3. Recommendation System:**

In [38]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [39]:
feat = pd.concat([df[['rating', 'members']], genre_encoded], axis=1)

In [40]:
cosine_sim = cosine_similarity(feat)

Recommendation Function

In [41]:
def recommend_anime(anime_id, cosine_sim,df):
    idx = df[df['anime_id'] == anime_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    anime_indices = [i[0] for i in sim_scores[1:11]]  # Get top 10 similar anime
    return df['anime_id'].iloc[anime_indices]

**4. Evaluation:**

**Training and testing of the data**

In [62]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [65]:
x= df.drop(['genre'],axis=1).values
y = df['genre'].values
X_train,X_test,Y_train,Y_test = train_test_split(x,y,random_state=0)


In [66]:
x

array([[32281, 'Kimi no Na wa.', 'Movie', '1', 0.9243697478991597,
        0.19786666403657144],
       [5114, 'Fullmetal Alchemist: Brotherhood', 'TV', '64',
        0.9111644657863145, 0.7827686025811095],
       [28977, 'Gintama°', 'TV', '51', 0.9099639855942379,
        0.11268314092543184],
       ...,
       [5621, 'Violence Gekiga David no Hoshi', 'OVA', '4',
        0.38535414165666265, 0.00020416113935723757],
       [6133, 'Violence Gekiga Shin David no Hoshi: Inma Densetsu',
        'OVA', '1', 0.397358943577431, 0.00016076456867260738],
       [26081, 'Yasuji no Pornorama: Yacchimae!!', 'Movie', '1',
        0.4549819927971189, 0.0001282171406591347]], dtype=object)

In [67]:
y

array([list(['Drama', ' Romance', ' School', ' Supernatural']),
       list(['Action', ' Adventure', ' Drama', ' Fantasy', ' Magic', ' Military', ' Shounen']),
       list(['Action', ' Comedy', ' Historical', ' Parody', ' Samurai', ' Sci-Fi', ' Shounen']),
       ..., list(['Hentai']), list(['Hentai']), list(['Hentai'])],
      dtype=object)

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3,random_state=10)

In [73]:
X_train


array([[744, 'Trouble Chocolate', 'TV', '20', 0.602641056422569,
        0.002697491382328719],
       [28701, 'Fate/stay night: Unlimited Blade Works 2nd Season', 'TV',
        '13', 0.8139255702280912, 0.20315019651742516],
       [8358, 'Crayon Shin-chan Movie 01: Action Kamen vs. Haigure Maou',
        'Movie', '1', 0.6482593037214885, 0.001959749680690005],
       ...,
       [5854, 'Shin Chou Kyou Ryo: Condor Hero III', 'TV', '26',
        0.709483793517407, 0.0013462799769209145],
       [27949,
        'Mattsu to Yanma to Moburi-san: Nanatsu no Hihou to Soratobu Oshiro',
        'ONA', '1', 0.4669867947178871, 0.0002081062821467494],
       [16287, 'One Piece: Romance Dawn', 'OVA', '1', 0.7118847539015607,
        0.008199979288000354]], dtype=object)

Evaluate the Recommendation System

In [74]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [75]:
def evaluate_recommendations(true_relevant, recommended):
    PS = precision_score(true_relevant, recommended)
    RS = recall_score(true_relevant, recommended)
    FS = f1_score(true_relevant, recommended)

    return PS, RS, FS

In [76]:
import numpy as np
TR= np.array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0])
recommended= np.array([1, 1, 0, 1, 0, 0, 1, 0, 0, 0])


In [77]:
PS,RS,FS = evaluate_recommendations(TR, recommended)

print(f"Precision Score : {PS:.2f}")
print(f"Recall Score : {RS:.2f}")
print(f"F1-score : {FS:.2f}")

Precision Score : 0.75
Recall Score : 0.60
F1-score : 0.67
